In [89]:
import pandas as pd
from pandas.api.types import is_string_dtype
import random
from matplotlib import rcParams
import matplotlib.pyplot as plt
import os
import time
import numpy as np
import glob

In [257]:
# define output file
Outputpath = 'D:/Powertac/finals-2021/data/'
Outputfilename = 'output' + '.csv'
Outputfilepath = Outputpath + Outputfilename

# find input .csv extracted by powertac logtools based on filename
path = 'D:/Powertac/finals-2021/data/'
number_games = list(range(1, 11))
#data_input = path + data_filename + '.csv'

# define logtool class used to extract game data
logtool_classes = ['MktPriceStats','WeatherForecastStats']

# create dataframe that will be filled by iterated files
data = []
df_main = pd.DataFrame(data)
df_main_w = pd.DataFrame(data)

# search for all files containing string in selected path
if os.path.isdir(path) == True:
    
    for logtool_class in logtool_classes:
        
        if logtool_class == 'MktPriceStats':
            for i in number_games:
                data_filename = '*mktPr' + str(i) + '.csv'       
                matching_files = glob.glob(path + data_filename)

                # loop for iterating over all files of one datatype (e.g. Market Prices)
                for matching_file in matching_files:
                    print(matching_file)

                    #df = pd.read_csv(data_input, header=None)
                    df = pd.read_csv(matching_file, header=None)
                    # create columns for cleared trades (CT)
                    df.columns = ['Timeslot', 'Weekday', 'Hour', 'CT1', 'CT2', 'CT3', 'CT4', 'CT5', 'CT6', 'CT7', 'CT8',\
                                  'CT9', 'CT10', 'CT11','CT12', 'CT13', 'CT14', 'CT15', 'CT16', 'CT17', 'CT18', 'CT19', \
                                  'CT20','CT21', 'CT22', 'CT23', 'CT24']
                    #df = df.set_index('Timeslot')

                    # split cleared trades into cleared amounts (CA) and cleared prices (CP)
                    c_index = 0
                    for column in df.columns[2:]:
                        if is_string_dtype(df[column]) == True:
                            df[column] = df[column].str.strip('[]').astype(str)
                            c_index = c_index + 1
                            CA = 'CA' + str(c_index)
                            CP = 'CP' + str(c_index)
                            df[[CA,CP]] = df[column].str.split(' ', expand = True)
                            df = df.drop(column, axis=1) 

                    # create new Dataframe for the prediction input
                    data = []
                    df1 = pd.DataFrame(data)
                    #df1.index.names = ['Timeslot']
                    df1 = pd.concat([df1, df.Timeslot], axis=1)
                    df1 = pd.concat([df1, df.Weekday], axis=1)
                    # fill with cleared trades
                    for column in df.columns[2:]:
                        df1 = pd.concat([df1, df[column]], axis=1)
                    # add more features:
                    # cleared trades on hour before
                    for column in df.columns[2:]:
                        name = column + '_H-1'
                        last_column = len(df1.columns)
                        df1.insert(len(df1.columns), name, df[column].shift(1))
                    # cleared trades on day before
                    for column in df.columns[2:]:
                        name = column + '_D-1'
                        last_column = len(df1.columns)
                        df1 = pd.concat([df1, df[column].shift(24).rename(name)], axis=1)  
                    # cleared trades on week before
                    for column in df.columns[2:]:
                        name = column + '_W-1'
                        last_column = len(df1.columns)
                        df1 = pd.concat([df1, df[column].shift(168).rename(name)], axis=1)

                    # delete rows with incomplete trades data
                    df1 = df1[df1['Timeslot'] > 552] # 385 (game start) + 168 (7*24)
                    df1 = df1[df1['Timeslot'] < 1300] # 385 (game start) + 168 (7*24)
                    #df1 = df1[df1.index > 552] # 385 (game start) + 168 (7*24)

                    # add rows to main dataframe
                    df_main = pd.concat([df_main, df1], ignore_index = True, axis = 0)

                
            
        elif logtool_class == 'WeatherForecastStats':
            data = []
            df1 = pd.DataFrame(data)
            df2 = pd.DataFrame(data)
            for i in number_games:
                data_filename = '*wthFrcst' + str(i) + '_' + '*.csv'  

                matching_files = glob.glob(path + data_filename)

                # loop for iterating over all files of one datatype (e.g. Market Prices)
                for matching_file in matching_files:
                    print(matching_file)

                    df = pd.read_csv(matching_file, header=None, delim_whitespace=True)
                    if matching_file.endswith('temp.csv'):
                        df.columns = ['Timeslot', 'TObs', 'TF1', 'TF2', 'TF3', 'TF4', 'TF5', 'TF6', 'TF7', 'TF8', 'TF9', \
                                      'TF10', 'TF11','TF12', 'TF13', 'TF14', 'TF15', 'TF16', 'TF17', 'TF18', \
                                      'TF19', 'TF20','TF21', 'TF22', 'TF23', 'TF24']
                        df = df.drop('TObs', axis=1)
                        # create new Dataframe for the prediction input      
                        df = df[df['Timeslot'] > 552] # 385 (game start) + 168 (7*24)
                        df = df[df['Timeslot'] < 1300] # 385 (game start) + 168 (7*24)
                        df1 = pd.concat([df1, df], ignore_index = True, axis = 0)
                    elif matching_file.endswith('wind.csv'):
                        df.columns = ['Timeslot','WSObs','WSF1', 'WSF2', 'WSF3', 'WSF4', 'WSF5', 'WSF6', 'WSF7', 'WSF8', 'WSF9', \
                                      'WSF10', 'WSF11','WSF12', 'WSF13', 'WSF14', 'WSF15', 'WSF16', 'WSF17', 'WSF18', \
                                      'WSF19', 'WSF20','WSF21', 'WSF22', 'WSF23', 'WSF24']
                        df = df.drop('WSObs', axis=1)
                        # create new Dataframe for the prediction input      
                        df = df[df['Timeslot'] > 552] # 385 (game start) + 168 (7*24)
                        df = df[df['Timeslot'] < 1300] # 385 (game start) + 168 (7*24)
                        df2 = pd.concat([df2, df], ignore_index = True, axis = 0)
            df1 = df1.drop(['Timeslot'], axis = 1)
            df2 = df2.drop(['Timeslot'], axis = 1)
            df_main = pd.concat([df_main, df1], axis = 1)  
            df_main = pd.concat([df_main, df2], axis = 1) 
            
        #df_main = pd.concat([df_main, df1], ignore_index = True)
            
    df_main['Weekday']=df_main['Timeslot'].mod(168)
    df_main['Timeslot']=df_main['Timeslot'].mod(24)

    # create output file from dataframe
    df_main.to_csv(Outputfilepath)
else:
    print('Path not existing.')

D:/Powertac/finals-2021/data\mktPr1.csv
D:/Powertac/finals-2021/data\mktPr2.csv
D:/Powertac/finals-2021/data\mktPr3.csv
D:/Powertac/finals-2021/data\mktPr4.csv
D:/Powertac/finals-2021/data\mktPr5.csv
D:/Powertac/finals-2021/data\mktPr6.csv
D:/Powertac/finals-2021/data\mktPr7.csv
D:/Powertac/finals-2021/data\mktPr8.csv
D:/Powertac/finals-2021/data\mktPr9.csv
D:/Powertac/finals-2021/data\mktPr10.csv
D:/Powertac/finals-2021/data\wthFrcst1_temp.csv
D:/Powertac/finals-2021/data\wthFrcst1_wind.csv
D:/Powertac/finals-2021/data\wthFrcst2_temp.csv
D:/Powertac/finals-2021/data\wthFrcst2_wind.csv
D:/Powertac/finals-2021/data\wthFrcst3_temp.csv
D:/Powertac/finals-2021/data\wthFrcst3_wind.csv
D:/Powertac/finals-2021/data\wthFrcst4_temp.csv
D:/Powertac/finals-2021/data\wthFrcst4_wind.csv
D:/Powertac/finals-2021/data\wthFrcst5_temp.csv
D:/Powertac/finals-2021/data\wthFrcst5_wind.csv
D:/Powertac/finals-2021/data\wthFrcst6_temp.csv
D:/Powertac/finals-2021/data\wthFrcst6_wind.csv
D:/Powertac/finals-2021

In [259]:
print(list(df_main.columns))

['Timeslot', 'Weekday', 'Hour', 'CA1', 'CP1', 'CA2', 'CP2', 'CA3', 'CP3', 'CA4', 'CP4', 'CA5', 'CP5', 'CA6', 'CP6', 'CA7', 'CP7', 'CA8', 'CP8', 'CA9', 'CP9', 'CA10', 'CP10', 'CA11', 'CP11', 'CA12', 'CP12', 'CA13', 'CP13', 'CA14', 'CP14', 'CA15', 'CP15', 'CA16', 'CP16', 'CA17', 'CP17', 'CA18', 'CP18', 'CA19', 'CP19', 'CA20', 'CP20', 'CA21', 'CP21', 'CA22', 'CP22', 'CA23', 'CP23', 'CA24', 'CP24', 'Hour_H-1', 'CA1_H-1', 'CP1_H-1', 'CA2_H-1', 'CP2_H-1', 'CA3_H-1', 'CP3_H-1', 'CA4_H-1', 'CP4_H-1', 'CA5_H-1', 'CP5_H-1', 'CA6_H-1', 'CP6_H-1', 'CA7_H-1', 'CP7_H-1', 'CA8_H-1', 'CP8_H-1', 'CA9_H-1', 'CP9_H-1', 'CA10_H-1', 'CP10_H-1', 'CA11_H-1', 'CP11_H-1', 'CA12_H-1', 'CP12_H-1', 'CA13_H-1', 'CP13_H-1', 'CA14_H-1', 'CP14_H-1', 'CA15_H-1', 'CP15_H-1', 'CA16_H-1', 'CP16_H-1', 'CA17_H-1', 'CP17_H-1', 'CA18_H-1', 'CP18_H-1', 'CA19_H-1', 'CP19_H-1', 'CA20_H-1', 'CP20_H-1', 'CA21_H-1', 'CP21_H-1', 'CA22_H-1', 'CP22_H-1', 'CA23_H-1', 'CP23_H-1', 'CA24_H-1', 'CP24_H-1', 'Hour_D-1', 'CA1_D-1', 'CP1_D-1'

In [249]:
df1

,TF1,TF2,TF3,TF4,TF5,TF6,TF7,TF8,TF9,TF10,...,TF15,TF16,TF17,TF18,TF19,TF20,TF21,TF22,TF23,TF24
0,-0.7,-0.7,-0.7,-0.7,-0.7,-0.7,-0.7,-0.7,-0.7,-0.7,...,-0.7,-0.7,-0.7,-0.7,-0.7,-0.7,-0.7,-0.7,-0.7,-0.7
1,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,...,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5
2,-2.1,-2.1,-2.1,-2.1,-2.1,-2.1,-2.1,-2.1,-2.1,-2.1,...,-2.1,-2.1,-2.1,-2.1,-2.1,-2.1,-2.1,-2.1,-2.1,-2.1
3,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,...,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5
4,-2.8,-2.8,-2.8,-2.8,-2.8,-2.8,-2.8,-2.8,-2.8,-2.8,...,-2.8,-2.8,-2.8,-2.8,-2.8,-2.8,-2.8,-2.8,-2.8,-2.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7465,30.9,30.9,30.6,30.6,30.6,30.6,30.6,30.6,30.6,30.6,...,30.6,30.6,30.6,30.6,30.6,30.6,30.6,30.6,30.6,30.9
7466,30.5,30.5,30.5,30.5,30.5,30.5,30.5,30.5,30.5,30.5,...,30.8,30.8,30.8,30.8,30.8,30.8,30.4,30.4,30.4,30.4
7467,29.3,29.1,29.1,29.1,29.2,29.2,29.2,29.2,29.2,29.2,...,29.2,29.2,29.2,29.2,29.2,29.2,29.2,29.2,29.2,29.2
7468,27.4,27.4,27.2,27.2,27.2,27.2,27.2,27.2,27.2,27.2,...,27.2,27.2,27.2,27.2,27.2,27.2,27.2,27.2,27.2,27.2


In [250]:
df2

,WSF1,WSF2,WSF3,WSF4,WSF5,WSF6,WSF7,WSF8,WSF9,WSF10,...,WSF15,WSF16,WSF17,WSF18,WSF19,WSF20,WSF21,WSF22,WSF23,WSF24
0,3.39,3.39,3.39,3.39,3.39,3.39,3.39,3.39,3.39,3.39,...,3.39,3.39,3.39,3.39,3.39,3.39,3.39,3.39,3.39,3.39
1,3.11,3.11,3.11,3.11,3.11,3.11,3.11,3.11,3.11,3.11,...,3.11,3.11,3.11,3.11,3.11,3.11,3.11,3.11,3.11,3.11
2,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,...,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98
3,3.07,3.07,3.07,3.07,3.07,3.07,3.07,3.07,3.07,3.07,...,3.07,3.07,3.07,3.07,3.07,3.07,3.07,3.07,3.07,3.07
4,3.25,3.25,3.25,3.25,3.25,3.25,3.25,3.25,3.25,3.25,...,3.25,3.25,3.25,3.25,3.25,3.25,3.25,3.25,3.25,3.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7465,3.13,3.13,3.20,3.20,3.20,3.23,3.23,3.23,3.21,3.21,...,3.22,3.22,3.22,3.22,3.22,3.22,3.22,3.22,3.22,3.13
7466,3.54,3.54,3.54,3.54,3.54,3.54,3.54,3.54,3.54,3.54,...,4.18,4.18,4.18,4.18,4.18,4.18,2.79,2.79,2.79,2.79
7467,3.26,2.53,2.53,2.53,2.50,2.50,2.50,2.49,2.49,2.49,...,2.49,2.49,2.50,2.50,2.50,2.50,2.50,2.50,2.50,2.50
7468,3.13,3.13,2.32,2.32,2.32,2.31,2.31,2.31,2.30,2.30,...,2.30,2.30,2.30,2.30,2.30,2.30,2.30,2.30,2.30,2.30


In [236]:
pd.concat([df1,df2], axis=1)

,Timeslot,TF1,TF2,TF3,TF4,TF5,TF6,TF7,TF8,TF9,...,WSF15,WSF16,WSF17,WSF18,WSF19,WSF20,WSF21,WSF22,WSF23,WSF24
0,553,-0.7,-0.7,-0.7,-0.7,-0.7,-0.7,-0.7,-0.7,-0.7,...,3.39,3.39,3.39,3.39,3.39,3.39,3.39,3.39,3.39,3.39
1,554,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,...,3.11,3.11,3.11,3.11,3.11,3.11,3.11,3.11,3.11,3.11
2,555,-2.1,-2.1,-2.1,-2.1,-2.1,-2.1,-2.1,-2.1,-2.1,...,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98
3,556,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,...,3.07,3.07,3.07,3.07,3.07,3.07,3.07,3.07,3.07,3.07
4,557,-2.8,-2.8,-2.8,-2.8,-2.8,-2.8,-2.8,-2.8,-2.8,...,3.25,3.25,3.25,3.25,3.25,3.25,3.25,3.25,3.25,3.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7465,1295,30.9,30.9,30.6,30.6,30.6,30.6,30.6,30.6,30.6,...,3.22,3.22,3.22,3.22,3.22,3.22,3.22,3.22,3.22,3.13
7466,1296,30.5,30.5,30.5,30.5,30.5,30.5,30.5,30.5,30.5,...,4.18,4.18,4.18,4.18,4.18,4.18,2.79,2.79,2.79,2.79
7467,1297,29.3,29.1,29.1,29.1,29.2,29.2,29.2,29.2,29.2,...,2.49,2.49,2.50,2.50,2.50,2.50,2.50,2.50,2.50,2.50
7468,1298,27.4,27.4,27.2,27.2,27.2,27.2,27.2,27.2,27.2,...,2.30,2.30,2.30,2.30,2.30,2.30,2.30,2.30,2.30,2.30


In [72]:
#pd.concat([df_main.iloc[:,2::-1], df_main.iloc[:,:2:-1]], axis=1)

,Hour,Weekday,Timeslot,CP24_W-1,CA24_W-1,CP23_W-1,CA23_W-1,CP22_W-1,CA22_W-1,CP21_W-1,...,CP5,CA5,CP4,CA4,CP3,CA3,CP2,CA2,CP1,CA1
191,2,5,554,27.8339,580.0204,17.4618,0.3608,17.406,0.1805,18.2356,...,41.3518,2.1805,41.6775,1.9291,47.7698,0.7287,47.5808,0.3374,31.5,1
192,3,5,555,34.0199,684.8622,26.9042,0.1624,18.2161,0.3035,22.751,...,36.3875,1.8362,36.3804,1.6558,41.5822,0.3036,0.5588,0.42,23.6907,0.6747
193,4,5,556,27.9432,631.0904,12.5633,0.2732,17.0184,0.3349,31.5,...,42.8871,1.5903,43.0187,1.2733,23.2905,0.3036,23.0291,0.6073,24.9755,0.5998
194,5,5,557,39.2402,815.6891,14.3327,0.3014,31.5,0.5503,22.6171,...,73.8727,0.2459,31.5,1.9295,40,0.5465,40.8333,0.5398,0.0,0.0
195,6,5,558,34.0146,699.1272,31.5,0.5859,12.002,0.0328,15.8693,...,51.0049,1,49.3955,1,51.0313,25.5529,0.0,0.0,31.6667,0.2999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1336,19,5,1699,45.3827,698.2115,45.8908,0.2339,40.4003,15.399,28.7304,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1337,20,5,1700,45.4872,763.8738,33.3654,0.4227,35.3226,0.0579,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1338,21,5,1701,33.6035,667.475,23.3361,0.0521,0.0,0.0,26.8568,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1339,22,5,1702,33.7286,618.9015,0.0,0.0,23.7621,0.6313,34.7792,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
df_main['Weekday']=df_main['Timeslot'].mod(168)
df_main['Timeslot']=df_main['Timeslot'].mod(24)
print(df_main)

      Timeslot  Weekday  Hour     CA1      CP1     CA2      CP2      CA3  \
191          2       50     2       1     31.5  0.3374  47.5808   0.7287   
192          3       51     3  0.6747  23.6907    0.42   0.5588   0.3036   
193          4       52     4  0.5998  24.9755  0.6073  23.0291   0.3036   
194          5       53     5     0.0      0.0  0.5398  40.8333   0.5465   
195          6       54     6  0.2999  31.6667     0.0      0.0  25.5529   
...        ...      ...   ...     ...      ...     ...      ...      ...   
1336        19       19    19     0.0      0.0     0.0      0.0      0.0   
1337        20       20    20     0.0      0.0     0.0      0.0      0.0   
1338        21       21    21     0.0      0.0     0.0      0.0      0.0   
1339        22       22    22     0.0      0.0     0.0      0.0      0.0   
1340        23       23    23     0.0      0.0     0.0      0.0      0.0   

          CP3     CA4  ... CA20_W-1 CP20_W-1 CA21_W-1 CP21_W-1 CA22_W-1  \
191   47.769